## Bias Mitigation using AIF360 - Heart Failure Prediction Dataset (Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)
Model Training and Evaluation

In [1]:
#load preprocessed data 
import pandas as pd
train_df = pd.read_csv("./data_subsets/train_25M_75F.csv")

X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

#check out the data
train_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,33,0,3,100.0,246.0,0,0,150.0,1,1.0,1,1
1,48,0,1,120.0,284.0,0,0,120.0,0,0.0,2,0
2,49,0,3,130.0,269.0,0,0,163.0,0,0.0,2,0
3,62,0,3,140.0,268.0,0,2,160.0,0,3.6,0,1
4,38,0,3,105.0,236.0,1,0,166.0,0,2.8,2,1


In [2]:
TARGET = "HeartDisease"
SENSITIVE = "Sex"   # 1 = Male, 0 = Female

categorical_cols = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']
continuous_cols  = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']

In [3]:
# Split train into X / y and keep sensitive feature for fairness evaluation
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

In [4]:
# scale numeric features only, fit on train, transform test
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_train[continuous_cols]),
    columns=continuous_cols, index=X_train.index
)
X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test[continuous_cols]),
    columns=continuous_cols, index=X_test.index
)

In [5]:
#one-hot encode categoricals; numeric are kept as is 
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)
ohe.fit(X_train[categorical_cols])

X_train_cat = pd.DataFrame(
    ohe.transform(X_train[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat = pd.DataFrame(
    ohe.transform(X_test[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

In [6]:
# Assemble final matrices
X_train_ready = pd.concat([X_train_cat, X_train_num_scaled], axis=1)
X_test_ready  = pd.concat([X_test_cat,  X_test_num_scaled],  axis=1)

print("Final feature shapes:", X_train_ready.shape, X_test_ready.shape)

Final feature shapes: (600, 18) (184, 18)


In [7]:
# create sensitive attribute arrays - after creating X_train_ready and X_test_ready
A_train = X_train["Sex"].astype(int).to_numpy().ravel()  # 1=Male, 0=Female
A_test  = X_test["Sex"].astype(int).to_numpy().ravel()

In [8]:
# setup for AIF360
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.base import clone
from IPython.display import display 

from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import ClassificationMetric

# Config 
protected_attr = "Sex"  # 1=Male, 0=Female
PRIV_VALUE = 1          # privileged = Male
label_name = "label"
favorable_label, unfavorable_label = 1, 0
privileged_groups   = [{protected_attr: PRIV_VALUE}]
unprivileged_groups = [{protected_attr: 1 - PRIV_VALUE}]

# Ensure 1-D ints for targets
y_train = np.asarray(y_train).astype(int).ravel()
y_test  = np.asarray(y_test).astype(int).ravel()

# Sensitive attribute arrays
A_train = X_train["Sex"].astype(int).to_numpy().ravel()
A_test  = X_test["Sex"].astype(int).to_numpy().ravel()

def _to_bld(y, A):
    y = (y.values if hasattr(y,'values') else np.asarray(y)).ravel()
    A = (A.values if hasattr(A,'values') else np.asarray(A)).ravel()
    df = pd.DataFrame({"dummy": np.zeros(len(y)), label_name: y, protected_attr: A})
    return BinaryLabelDataset(
        df=df,
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

def fair_metrics(y_true, y_pred, A, y_scores=None, absolute=True):
    """AIF360-based DP and EO (equal opportunity) differences."""
    t = _to_bld(y_true, A)
    p = _to_bld(y_pred, A)
    if y_scores is not None:
        p.scores = np.asarray(y_scores).reshape(-1, 1)
    cm = ClassificationMetric(
        t, p,
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups
    )
    dp = cm.statistical_parity_difference()
    eo = cm.equal_opportunity_difference()
    return (abs(dp), abs(eo)) if absolute else (dp, eo)

def get_scores(model, X):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    if hasattr(model, "decision_function"):
        z = model.decision_function(X)
        return (z - z.min()) / (z.max() - z.min() + 1e-12)
    return model.predict(X).astype(float)

def selection_rate(y_pred, positive=1):
    y_pred = np.asarray(y_pred).ravel()
    return np.mean(y_pred == positive)

def per_group_table(y_true, y_pred, A, positive=1, group_name="Sex"):
    """Keeps your existing API (positive=...), uses sklearn metrics."""
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    A = np.asarray(A).ravel()
    rows = []
    for g in np.unique(A):
        idx = (A == g)
        yt, yp = y_true[idx], y_pred[idx]
        tn, fp, fn, tp = confusion_matrix(yt, yp, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) else 0.0
        fpr = fp / (fp + tn) if (fp + tn) else 0.0
        rec = recall_score(yt, yp, pos_label=positive)   # equals TPR for binary
        sr  = selection_rate(yp, positive=positive)
        acc = accuracy_score(yt, yp)
        rows.append({group_name: g, "TPR": tpr, "FPR": fpr,
                     "Recall": rec, "SelectionRate": sr, "Accuracy": acc})
    return pd.DataFrame(rows).set_index(group_name)

def aif_diffs(y_true, y_pred, A, *, abs_vals=True):
    """Alternative disparities (AIF360): DP and average odds difference."""
    t = _to_bld(y_true, A)
    p = _to_bld(y_pred, A)
    cm = ClassificationMetric(
        t, p,
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups
    )
    dp = cm.statistical_parity_difference()
    eo = cm.average_odds_difference()   # avg of TPR/FPR diffs
    if abs_vals:
        dp, eo = abs(dp), abs(eo)
    return dp, eo

def print_row(title, acc, dp, eo, note=""):
    print(f"{title:>24s} | Acc {acc:.4f} | DP {dp:.4f} | EO {eo:.4f} {('|' if note else '')} {note}")

# to print a model cleanly (fixed call sites)
def report_model(name, y_true, y_pred, A, scores=None, note=""):
    acc = accuracy_score(y_true, y_pred)
    dp, eo = fair_metrics(y_true, y_pred, A, y_scores=scores, absolute=True)  # no pos_label here
    tbl = per_group_table(y_true, y_pred, A, positive=favorable_label, group_name="Sex").round(6)
    
    print(f"\n=== {name} ===")
    display(tbl)
    print(f"Overall -> Accuracy: {acc:.4f} | DP diff: {dp:.4f} | EO diff: {eo:.4f}"
          + (f" | {note}" if note else ""))
    
    return {"Model": name, "Accuracy": acc, "DP diff": dp, "EO diff": eo}

# Pre: compute reweighing weights ONCE on TRAIN
_bld_train = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_train_ready),
                  pd.Series(y_train, name=label_name),
                  pd.Series(A_train, name=protected_attr)], axis=1),
    label_names=[label_name],
    protected_attribute_names=[protected_attr],
    favorable_label=favorable_label,
    unfavorable_label=unfavorable_label
)
_rw = Reweighing(unprivileged_groups=unprivileged_groups,
                 privileged_groups=privileged_groups).fit(_bld_train)
_rw_weights = _rw.transform(_bld_train).instance_weights.ravel()

# Turn weights into a resampled training set
def resample_by_weights(X, y, A, weights, n_samples=None, random_state=42):
    rng = np.random.default_rng(random_state)
    Xn = np.asarray(X); yn = np.asarray(y).ravel(); An = np.asarray(A).ravel()
    w = np.clip(np.asarray(weights, dtype=float), 1e-12, None)
    p = w / w.sum()
    n = n_samples or len(yn)
    idx = rng.choice(len(yn), size=n, replace=True, p=p)
    return Xn[idx], yn[idx], An[idx]

Xrw, yrw, Arw = resample_by_weights(
    X_train_ready, y_train, A_train, _rw_weights,
    n_samples=len(y_train), random_state=42
)

# Post: make a small TRAIN-based calibration split (no test leakage)
trn_X, cal_X, trn_y, cal_y, trn_A, cal_A = train_test_split(
    X_train_ready, y_train, A_train, test_size=0.12, stratify=y_train, random_state=42
)

# Make types consistent to avoid the PCA warning 
X_test_np = np.asarray(X_test_ready)
trn_X_np  = np.asarray(trn_X)
cal_X_np  = np.asarray(cal_X)

C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; 

In [9]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

### Traditional ML Models - Baseline: K-Nearest Neighbors (KNN) & Decision Tree (DT)

In [10]:
#import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

#define a function 
def evaluate_model(y_true, y_pred, model_name):
    print(f"=== {model_name} Evaluation ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='binary'))
    print("Recall   :", recall_score(y_true, y_pred, average='binary'))
    print("F1 Score :", f1_score(y_true, y_pred, average='binary'))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n" + "="*40 + "\n")

### PCA-KNN

In [11]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

#1) PCA + KNN pipeline (on one-hot encoded + scaled features)
pca_knn = Pipeline([
    ('pca', PCA(n_components=0.95, random_state=42)),  # keep 95% variance
    ('knn', KNeighborsClassifier(
        n_neighbors=15, metric='manhattan', weights='distance'
    ))
])

pca_knn.fit(X_train_ready, y_train)

# Inspect PCA details
n_comp = pca_knn.named_steps['pca'].n_components_
expl_var = pca_knn.named_steps['pca'].explained_variance_ratio_.sum()
print(f"PCA components: {n_comp} | Explained variance retained: {expl_var:.3f}")

# 2) Evaluate 
y_pred_pca_knn = pca_knn.predict(X_test_ready)
probs_pca_knn = pca_knn.predict_proba(X_test_ready)[:, 1]
  
evaluate_model(y_test, y_pred_pca_knn, "KNN (best params)")

PCA components: 11 | Explained variance retained: 0.952
=== KNN (best params) Evaluation ===
Accuracy : 0.8315217391304348
Precision: 0.9080459770114943
Recall   : 0.7745098039215687
F1 Score : 0.8359788359788359

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.90      0.83        82
           1       0.91      0.77      0.84       102

    accuracy                           0.83       184
   macro avg       0.84      0.84      0.83       184
weighted avg       0.84      0.83      0.83       184

Confusion Matrix:
 [[74  8]
 [23 79]]




### Tuned Decision Tree (DT)

In [12]:
# Alternative DT tuning focused on higher recall
# Changes vs previous:
#  - Remove calibration (predict uses raw tree probs at 0.5)
#  - Tune class_weight (heavier positive weights allowed)
#  - Broaden depth a bit but keep regularization via min_samples_* and tiny impurity decrease
#  - Prune only with very small ccp_alphas to avoid killing recall

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

# Simpler-but-expressive trees + tuned class weights
base_dt = DecisionTreeClassifier(random_state=42)

param_grid_simple = {
    "criterion": ["gini", "entropy"],                  # add "log_loss" if your sklearn supports it
    "max_depth": [4, 5, 6, 7, 8, 9, 10],               # a bit deeper to help recall
    "min_samples_split": [5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 6],
    "min_impurity_decrease": [0.0, 1e-4, 1e-3],
    "class_weight": ["balanced", {0:1,1:2}, {0:1,1:3}, {0:1,1:4}],  # stronger push toward positives
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)

grid_simple = GridSearchCV(
    estimator=base_dt,
    param_grid=param_grid_simple,
    cv=cv,
    scoring="recall",      # prioritize sensitivity for class 1
    n_jobs=-1,
    verbose=0,
    refit=True
)
grid_simple.fit(X_train_ready, y_train)

best_params = grid_simple.best_params_
print("Stage A — Best DT params:", best_params)
print("Stage A — Best CV Recall:", round(grid_simple.best_score_, 4))

# Train a zero-pruned model with best params to get the pruning path
dt0 = DecisionTreeClassifier(random_state=42, **best_params, ccp_alpha=0.0).fit(X_train_ready, y_train)


# Stage B — Gentle cost-complexity pruning (favor small alphas)
path = dt0.cost_complexity_pruning_path(X_train_ready, y_train)
ccp_alphas = path.ccp_alphas

# Focus on tiny alphas only + 0.0 to avoid big recall loss
small_slice = ccp_alphas[: min(30, len(ccp_alphas))]  # first 30 values are typically the smallest
candidate_alphas = np.unique(np.r_[0.0, small_slice])

cv_scores = []
for alpha in candidate_alphas:
    dt_alpha = DecisionTreeClassifier(random_state=42, **best_params, ccp_alpha=alpha)
    rec = cross_val_score(dt_alpha, X_train_ready, y_train, cv=cv, scoring="recall", n_jobs=-1).mean()
    cv_scores.append((alpha, rec))

best_alpha, best_cv_recall = max(cv_scores, key=lambda x: x[1])
print(f"Stage B — Best ccp_alpha: {best_alpha:.6f} | CV Recall: {best_cv_recall:.4f}")

alt_best_dt = DecisionTreeClassifier(random_state=42, **best_params, ccp_alpha=best_alpha).fit(X_train_ready, y_train)


# Evaluation
y_pred = alt_best_dt.predict(X_test_ready)               
y_prob = alt_best_dt.predict_proba(X_test_ready)[:, 1]   

evaluate_model(y_test, y_pred, "Alternative Tuned & Pruned Decision Tree")

Stage A — Best DT params: {'class_weight': {0: 1, 1: 4}, 'criterion': 'entropy', 'max_depth': 4, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 4, 'min_samples_split': 5}
Stage A — Best CV Recall: 0.9733
Stage B — Best ccp_alpha: 0.000000 | CV Recall: 0.9733
=== Alternative Tuned & Pruned Decision Tree Evaluation ===
Accuracy : 0.8260869565217391
Precision: 0.7966101694915254
Recall   : 0.9215686274509803
F1 Score : 0.8545454545454545

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.71      0.78        82
           1       0.80      0.92      0.85       102

    accuracy                           0.83       184
   macro avg       0.84      0.81      0.82       184
weighted avg       0.83      0.83      0.82       184

Confusion Matrix:
 [[58 24]
 [ 8 94]]




### Ensemble Model - Random Forest (RF)

In [13]:
# Random Forest: hyperparameter tuning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV


# 1) GridSearchCV over impactful RF params
rf = RandomForestClassifier(random_state=42)

param_grid = {
    "n_estimators": [200, 400, 600],
    "max_depth": [None, 8, 12, 16],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.8],  # 0.8 = 80% of features
    "class_weight": [None, "balanced"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=cv,
    scoring="recall",          
    n_jobs=-1,
    verbose=1,
    refit=True
)

grid.fit(X_train_ready, y_train)
best_rf = grid.best_estimator_
print("Best RF params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)

# 2) Evaluate best RF 
y_pred = best_rf.predict(X_test_ready)
y_prob = best_rf.predict_proba(X_test_ready)[:, 1]
evaluate_model(y_test, y_pred, "Tuned Random Forest")

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best RF params: {'class_weight': None, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Best CV F1: 0.9566666666666667
=== Tuned Random Forest Evaluation ===
Accuracy : 0.842391304347826
Precision: 0.9010989010989011
Recall   : 0.803921568627451
F1 Score : 0.8497409326424871

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.89      0.83        82
           1       0.90      0.80      0.85       102

    accuracy                           0.84       184
   macro avg       0.84      0.85      0.84       184
weighted avg       0.85      0.84      0.84       184

Confusion Matrix:
 [[73  9]
 [20 82]]




### Deep Learning - Multi-layer Perceptron

In [14]:
#import required library 
from sklearn.neural_network import MLPClassifier

### Adam MLP + Early Stopping

In [15]:
#Adam + Early Stopping 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

adammlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),   # slightly smaller/deeper can help
    activation='relu',
    solver='adam',
    learning_rate_init=1e-3,       # smaller step can stabilize
    alpha=1e-3,                    # L2 regularization to reduce overfitting
    batch_size=32,
    max_iter=1000,                 # increased max_iter
    early_stopping=True,           # use a validation split internally
    validation_fraction=0.15,
    n_iter_no_change=25,          
    tol=1e-4,
    random_state=42
)

adammlp.fit(X_train_ready, y_train)  
y_pred_mlp = adammlp.predict(X_test_ready)                     
y_prob_mlp = adammlp.predict_proba(X_test_ready)[:, 1]         

evaluate_model(y_test, y_pred_mlp, "(Adam + EarlyStopping)")

=== (Adam + EarlyStopping) Evaluation ===
Accuracy : 0.8206521739130435
Precision: 0.8709677419354839
Recall   : 0.7941176470588235
F1 Score : 0.8307692307692308

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.85      0.81        82
           1       0.87      0.79      0.83       102

    accuracy                           0.82       184
   macro avg       0.82      0.82      0.82       184
weighted avg       0.83      0.82      0.82       184

Confusion Matrix:
 [[70 12]
 [21 81]]




### Bias Mitigation AIF360

In [16]:
# Setup: install AIF360
# Uncomment the next line if running locally for the first time
#!pip install aif360

In [17]:
import aif360
print("AIF360 version:", aif360.__version__)

AIF360 version: 0.6.1


### Bias Mitigation AIF 360 - KNN 

In [18]:
#get the Fairlearn KNN Baseline for AIF360 bias mitigation
knn_base = pca_knn

yhat_knn_base   = knn_base.predict(X_test_ready)         
scores_knn_base = get_scores(knn_base, X_test_ready)

res_knn_base = report_model("Fairlearn - KNN baseline", y_test, yhat_knn_base, A_test, scores=scores_knn_base)


#Pre (Reweighing)
knn_pre        = clone(pca_knn).fit(Xrw, yrw)
yhat_knn_pre   = knn_pre.predict(X_test_ready)
scores_knn_pre = get_scores(knn_pre, X_test_ready)
res_knn_pre    = report_model("KNN pre: Reweigh",
                              y_test, yhat_knn_pre, A_test,
                              scores=scores_knn_pre,
                              note="resampled by AIF360 weights")

#Post (Equalized Odds)
cal_scores_knn   = get_scores(knn_base, cal_X_np)  # baseline KNN on CAL
post_knn = EqOddsPostprocessing(privileged_groups=privileged_groups,
                                unprivileged_groups=unprivileged_groups)
post_knn.fit(_to_bld(cal_y, cal_A),
             _to_bld((cal_scores_knn >= 0.5).astype(int), cal_A))

pred_knn_post_bld = post_knn.predict(_to_bld((scores_knn_base >= 0.5).astype(int), A_test))
yhat_knn_post     = pred_knn_post_bld.labels.ravel().astype(int)

res_knn_post = report_model("KNN post: EqOdds",
                            y_test, yhat_knn_post, A_test,
                            scores=scores_knn_base,
                            note="calibrated on held-out TRAIN")


=== Fairlearn - KNN baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.03125,0.666667,0.131579,0.921053
1,0.781250,0.14000,0.781250,0.561644,0.808219


Overall -> Accuracy: 0.8315 | DP diff: 0.4301 | EO diff: 0.1146

=== KNN pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.125,0.833333,0.236842,0.868421
1,0.812500,0.180,0.812500,0.595890,0.815068


Overall -> Accuracy: 0.8261 | DP diff: 0.3590 | EO diff: 0.0208 | resampled by AIF360 weights

=== KNN post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.03125,0.666667,0.131579,0.921053
1,0.781250,0.14000,0.781250,0.561644,0.808219


Overall -> Accuracy: 0.8315 | DP diff: 0.4301 | EO diff: 0.1146 | calibrated on held-out TRAIN


## KNN + AIF360

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|------------------:|------:|
| Baseline            | 0.8315   | 0.4301  | 0.1146            | 0.5447 |
| Pre: Reweigh        | 0.8261   | **0.3590** | **0.0208**     | **0.3798** |
| Post: EqualizedOdds | 0.8315   | 0.4301  | 0.1146            | 0.5447 |

---

### Per-group behavior (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.132**, 1 **0.562** → large DP gap (**0.430**).  
- **TPR (Recall):** 0 **0.667**, 1 **0.781** → EO gap **0.115** (males recalled more).  
- **FPR:** 0 **0.031**, 1 **0.140** (males have higher false positives).  
- **Note:** Accuracy is good, but fairness disparities remain significant.

#### Pre-processing: Reweigh
- **Selection rate:** 0 **0.237**, 1 **0.596** → DP improves to **0.359**.  
- **TPR (Recall):** 0 **0.833**, 1 **0.813** → **near-parity recall** (**EO 0.021**, best).  
- **FPR:** 0 **0.125**, 1 **0.180** (both ↑).  
- **Note:** Achieves the best fairness (lowest DP+EO), but with a modest accuracy drop.

#### Post-processing: Equalized Odds
- **Selection rate:** 0 **0.132**, 1 **0.562** → DP gap **0.430** (unchanged).  
- **TPR (Recall):** 0 **0.667**, 1 **0.781** → EO gap unchanged (**0.115**).  
- **FPR:** 0 **0.031**, 1 **0.140** (same as baseline).  
- **Note:** Results are essentially identical to baseline — no fairness gains.

---

### Implications
- **Most fair overall:** **Pre: Reweigh** — lowest combined disparity (DP+EO ≈ 0.380), by nearly eliminating the recall gap and reducing DP.  
- **Baseline** has decent accuracy but strong fairness gaps.  
- **Post: EqOdds** fails to improve fairness compared to baseline.  

---

In [19]:
#get the Fairlearn DT Baseline for AIF360 bias mitigation
dt_base = alt_best_dt

yhat_dt_base   = dt_base.predict(X_test_ready)         
scores_dt_base = get_scores(dt_base, X_test_ready)

res_dt_base = report_model("Fairlearn - DT baseline", y_test, yhat_dt_base, A_test, scores=scores_dt_base)

# Pre (Reweighing)
dt_pre = clone(alt_best_dt).fit(Xrw, yrw)
yhat_dt_pre = dt_pre.predict(X_test_np)
scores_dt_pre = get_scores(dt_pre, X_test_np)
_ = report_model("DT pre: Reweigh", y_test, yhat_dt_pre, A_test, scores=scores_dt_pre,
                 note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores_dt = get_scores(dt_base, cal_X_np)
post_dt = EqOddsPostprocessing(privileged_groups=privileged_groups,
                               unprivileged_groups=unprivileged_groups)
post_dt.fit(_to_bld(cal_y, cal_A),
            _to_bld((cal_scores_dt >= 0.5).astype(int), cal_A))
yhat_dt_post = post_dt.predict(_to_bld((scores_dt_base >= 0.5).astype(int), A_test)).labels.ravel().astype(int)
_ = report_model("DT post: EqOdds", y_test, yhat_dt_post, A_test, scores=scores_dt_base,
                 note="calibrated on held-out TRAIN")


=== Fairlearn - DT baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.28125,0.833333,0.368421,0.736842
1,0.927083,0.30000,0.927083,0.712329,0.849315


Overall -> Accuracy: 0.8261 | DP diff: 0.3439 | EO diff: 0.0938

=== DT pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.15625,0.666667,0.236842,0.815789
1,0.927083,0.26000,0.927083,0.698630,0.863014


Overall -> Accuracy: 0.8533 | DP diff: 0.4618 | EO diff: 0.2604 | resampled by AIF360 weights

=== DT post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.34375,0.833333,0.421053,0.684211
1,0.927083,0.30000,0.927083,0.712329,0.849315


Overall -> Accuracy: 0.8152 | DP diff: 0.2913 | EO diff: 0.0938 | calibrated on held-out TRAIN


## DT + AIF360

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|------------------:|------:|
| Baseline            | 0.8261   | 0.3439  | **0.0938**        | **0.4377** |
| Pre: Reweigh        | 0.8533   | 0.4618  | 0.2604            | 0.7222 |
| Post: EqualizedOdds | 0.8152   | **0.2913** | **0.0938**     | 0.3851 |

---

### Per-group behavior (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.368**, 1 **0.712** → DP gap **0.344**.  
- **TPR (Recall):** 0 **0.833**, 1 **0.927** → EO gap **0.094** (small).  
- **FPR:** 0 **0.281**, 1 **0.300** (males slightly higher).  
- **Note:** Good accuracy with minimal EO disparity, but a sizeable DP gap.

#### Pre-processing: Reweigh
- **Selection rate:** 0 **0.237**, 1 **0.699** → DP worsens to **0.462**.  
- **TPR (Recall):** 0 **0.667**, 1 **0.927** → EO worsens to **0.260** (large).  
- **FPR:** 0 **0.156**, 1 **0.260**.  
- **Note:** Accuracy improves (0.853), but fairness degrades strongly (both DP and EO worsen).

#### Post-processing: Equalized Odds
- **Selection rate:** 0 **0.421**, 1 **0.712** → DP improves to **0.291**.  
- **TPR (Recall):** 0 **0.833**, 1 **0.927** → EO parity remains **0.094**.  
- **FPR:** 0 **0.344**, 1 **0.300** (female ↑).  
- **Note:** Accuracy slightly lower (0.815), but fairness metrics (DP+EO) are the best overall.

---

### Implications
- **Most fair overall:** **Post: EqOdds**, with the lowest combined gap (DP+EO ≈ 0.385), balancing DP and EO reasonably well.  
- **Baseline** performs well on EO (≈0.094) with solid accuracy but suffers from higher DP disparity.  
- **Pre: Reweigh** increases accuracy the most but worsens both DP and EO, making it the least fair option.  
- **Takeaway:** For DT, post-processing gives the best fairness–performance trade-off, while baseline is competitive due to its strong EO parity.

---

In [21]:
# Random Forest (RF) with AIF360

# -get Fairlearn baseline
yhat_rf_base    = best_rf.predict(X_test_ready)
scores_rf_base  = get_scores(best_rf, X_test_ready)
res_rf_base     = report_model("Fairlearn - RF baseline", y_test, yhat_rf_base, A_test, scores=scores_rf_base)

# Pre (Reweighing via sample_weight)
rf_pre          = clone(best_rf).fit(X_train_ready, y_train, sample_weight=_rw_weights)
yhat_rf_pre     = rf_pre.predict(X_test_ready)
scores_rf_pre   = get_scores(rf_pre, X_test_ready)
res_rf_pre      = report_model("RF pre: Reweigh (sample_weight)",
                               y_test, yhat_rf_pre, A_test, scores=scores_rf_pre)

# Post (Equalized Odds) learned on CAL
cal_scores_rf   = get_scores(best_rf, cal_X_np)  # baseline rf on calibration split
post_rf = EqOddsPostprocessing(privileged_groups=privileged_groups,
                               unprivileged_groups=unprivileged_groups)
post_rf.fit(_to_bld(cal_y, cal_A),
            _to_bld((cal_scores_rf >= 0.5).astype(int), cal_A))

pred_rf_post_bld = post_rf.predict(_to_bld((scores_rf_base >= 0.5).astype(int), A_test))
yhat_rf_post     = pred_rf_post_bld.labels.ravel().astype(int)

res_rf_post = report_model("RF post: EqOdds",
                           y_test, yhat_rf_post, A_test,
                           scores=scores_rf_base,
                           note="calibrated on held-out TRAIN")



=== Fairlearn - RF baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.03125,0.833333,0.157895,0.947368
1,0.802083,0.16000,0.802083,0.582192,0.815068


Overall -> Accuracy: 0.8424 | DP diff: 0.4243 | EO diff: 0.0312

=== RF pre: Reweigh (sample_weight) ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.03125,0.833333,0.157895,0.947368
1,0.802083,0.16000,0.802083,0.582192,0.815068


Overall -> Accuracy: 0.8424 | DP diff: 0.4243 | EO diff: 0.0312

=== RF post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.03125,0.833333,0.157895,0.947368
1,0.802083,0.16000,0.802083,0.582192,0.815068


Overall -> Accuracy: 0.8424 | DP diff: 0.4243 | EO diff: 0.0312 | calibrated on held-out TRAIN


## RF + AIF360 

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|------------------:|------:|
| Baseline            | 0.8424   | 0.4243  | **0.0312**        | **0.4555** |
| Pre: Reweigh        | 0.8424   | 0.4243  | **0.0312**        | **0.4555** |
| Post: EqualizedOdds | 0.8424   | 0.4243  | **0.0312**        | **0.4555** |

---

### Per-group behavior (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.158**, 1 **0.582** → DP gap **0.424** (large).  
- **TPR (Recall):** 0 **0.833**, 1 **0.802** → EO gap **0.031** (very small).  
- **FPR:** 0 **0.031**, 1 **0.160** (males higher).  
- **Note:** High accuracy (0.842), excellent EO parity, but strong selection disparity.

#### Pre-processing: Reweigh
- **Selection rate, TPR, FPR:** identical to baseline.  
- **Note:** No changes — fairness metrics and accuracy remain the same as baseline.

#### Post-processing: Equalized Odds
- **Selection rate, TPR, FPR:** identical to baseline.  
- **Note:** Again, no improvement — results are the same as baseline, indicating EqOdds had no effect here.

---

### Implications
- **Most fair overall:** **Baseline** — already strong EO (0.031) with high accuracy.  
- **Pre: Reweigh** and **Post: EqOdds** made no difference in this setup.  
- **Takeaway:** RF’s baseline model is already optimal in balancing fairness and accuracy, though the DP disparity (0.424) remains a persistent issue.

---

In [22]:
#get the Fairlearn MLP Baseline for AIF360 bias mitigation
mlp_base = adammlp
yhat_mlp_base   = mlp_base.predict(X_test_ready)         
scores_mlp_base = get_scores(mlp_base, X_test_ready)

res_mlp_base = report_model("Fairlearn - MLP baseline", y_test, yhat_mlp_base, A_test, scores=scores_mlp_base)

# Pre (Reweighing)
mlp_pre = clone(adammlp).fit(Xrw, yrw)
yhat_mlp_pre = mlp_pre.predict(X_test_np)
scores_mlp_pre = get_scores(mlp_pre, X_test_np)
_ = report_model("MLP pre: Reweigh", y_test, yhat_mlp_pre, A_test, scores=scores_mlp_pre,
                 note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores_mlp = get_scores(mlp_base, cal_X_np)
post_mlp = EqOddsPostprocessing(privileged_groups=privileged_groups,
                                unprivileged_groups=unprivileged_groups)
post_mlp.fit(_to_bld(cal_y, cal_A),
             _to_bld((cal_scores_mlp >= 0.5).astype(int), cal_A))
yhat_mlp_post = post_mlp.predict(_to_bld((scores_mlp_base >= 0.5).astype(int), A_test)).labels.ravel().astype(int)
_ = report_model("MLP post: EqOdds", y_test, yhat_mlp_post, A_test, scores=scores_mlp_base,
                 note="calibrated on held-out TRAIN")


=== Fairlearn - MLP baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.03125,0.666667,0.131579,0.921053
1,0.802083,0.22000,0.802083,0.602740,0.794521


Overall -> Accuracy: 0.8207 | DP diff: 0.4712 | EO diff: 0.1354

=== MLP pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.09375,0.666667,0.184211,0.868421
1,0.802083,0.26000,0.802083,0.616438,0.780822


Overall -> Accuracy: 0.7989 | DP diff: 0.4322 | EO diff: 0.1354 | resampled by AIF360 weights

=== MLP post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.1875,0.833333,0.289474,0.815789
1,0.802083,0.2200,0.802083,0.602740,0.794521


Overall -> Accuracy: 0.7989 | DP diff: 0.3133 | EO diff: 0.0312 | calibrated on held-out TRAIN


## MLP + AIF360

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|------------------:|------:|
| Baseline            | 0.8207   | 0.4712  | 0.1354            | 0.6066 |
| Pre: Reweigh        | 0.7989   | 0.4322  | 0.1354            | 0.5676 |
| Post: EqualizedOdds | 0.7989   | **0.3133** | **0.0312**     | **0.3445** |

---

### Per-group behavior (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.132**, 1 **0.603** → DP gap **0.471** (large).  
- **TPR (Recall):** 0 **0.667**, 1 **0.802** → EO gap **0.135**.  
- **FPR:** 0 **0.031**, 1 **0.220** (much higher for males).  
- **Note:** Highest accuracy (0.821) but worst fairness, with both DP and EO disparities evident.

#### Pre-processing: Reweigh
- **Selection rate:** 0 **0.184**, 1 **0.616** → DP improves slightly to **0.432**.  
- **TPR (Recall):** 0 **0.667**, 1 **0.802** → EO gap unchanged (**0.135**).  
- **FPR:** 0 **0.094**, 1 **0.260** (↑ vs baseline).  
- **Note:** Small DP improvement, EO unchanged, and accuracy drops modestly.

#### Post-processing: Equalized Odds
- **Selection rate:** 0 **0.289**, 1 **0.603** → **best DP** (**0.313**).  
- **TPR (Recall):** 0 **0.833**, 1 **0.802** → **best EO** (**0.031**, near parity).  
- **FPR:** 0 **0.188** (↑ sharply), 1 **0.220** (≈ baseline).  
- **Note:** Strongest fairness improvement (lowest DP+EO), though accuracy falls further to ~0.799.

---

### Implications
- **Most fair overall:** **Post: EqOdds** (DP+EO ≈ 0.345) — drastically reduces both DP and EO, though at the cost of accuracy.  
- **Pre: Reweigh** provides only modest DP gains with unchanged EO.  
- **Baseline** offers the best accuracy but suffers from the highest disparities.  

---

First fairness mitigation: pre- and post-processing was performed on the designated best performing models (KNN, DT, RF, MLP) for CVD prediction.  In addition, these results are compared to a fairness-aware in-processing model - Adversarial Debiasing offered by AIF360.

In [23]:
#Adversarial Debiasing - In-processing by AIF360
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import tensorflow as tf
    from aif360.algorithms.inprocessing import AdversarialDebiasing

    # TF1 graph mode - required by AIF360's implementation 
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    # Build AIF360 datasets with FEATURES + label + sensitive attribute
    bld_tr = BinaryLabelDataset(
        df=pd.concat([
            pd.DataFrame(X_train_ready).reset_index(drop=True),
            pd.Series(y_train, name=label_name),
            pd.Series(A_train, name=protected_attr)
        ], axis=1),
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

    bld_te = BinaryLabelDataset(
        df=pd.concat([
            pd.DataFrame(X_test_ready).reset_index(drop=True),
            pd.Series(y_test, name=label_name),
            pd.Series(A_test, name=protected_attr)
        ], axis=1),
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

    # Train + predict inside a TF1 session
    sess = tf.compat.v1.Session()
    with sess.as_default():
        adv = AdversarialDebiasing(
            privileged_groups=privileged_groups,
            unprivileged_groups=unprivileged_groups,
            scope_name="adv_debias",
            debias=True,
            sess=sess
        )
        adv.fit(bld_tr)
        pred_te = adv.predict(bld_te)

        # Extract labels and (if available) scores
        yhat_adv = pred_te.labels.ravel().astype(int)
        scores_adv = getattr(pred_te, "scores", None)
        if scores_adv is None:
            scores_adv = yhat_adv.astype(float)

    # Clean up TF graph
    tf.compat.v1.reset_default_graph()
    sess.close()

    # Same structured output as other models
    _ = report_model(
        "ADV in-proc (AIF360)",
        y_test, yhat_adv, A_test,
        scores=scores_adv,
        note="trained on X_train_ready"
    )

except Exception as e:
    print("AdversarialDebiasing skipped:", type(e).__name__, e)



epoch 0; iter: 0; batch classifier loss: 0.733708; batch adversarial loss: 0.548406
epoch 1; iter: 0; batch classifier loss: 0.683735; batch adversarial loss: 0.593069
epoch 2; iter: 0; batch classifier loss: 0.622917; batch adversarial loss: 0.580076
epoch 3; iter: 0; batch classifier loss: 0.586533; batch adversarial loss: 0.633904
epoch 4; iter: 0; batch classifier loss: 0.504032; batch adversarial loss: 0.540876
epoch 5; iter: 0; batch classifier loss: 0.501094; batch adversarial loss: 0.565062
epoch 6; iter: 0; batch classifier loss: 0.485903; batch adversarial loss: 0.589199
epoch 7; iter: 0; batch classifier loss: 0.452581; batch adversarial loss: 0.565060
epoch 8; iter: 0; batch classifier loss: 0.473150; batch adversarial loss: 0.603425
epoch 9; iter: 0; batch classifier loss: 0.404911; batch adversarial loss: 0.551220
epoch 10; iter: 0; batch classifier loss: 0.386041; batch adversarial loss: 0.599141
epoch 11; iter: 0; batch classifier loss: 0.466139; batch adversarial los

,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.0625,0.666667,0.157895,0.894737
1,0.864583,0.2400,0.864583,0.650685,0.828767


Overall -> Accuracy: 0.8424 | DP diff: 0.4928 | EO diff: 0.1979 | trained on X_train_ready


## ADV In-processing (AIF360)

### Results overview
| Variant        | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|----------------|---------:|--------:|------------------:|------:|
| ADV in-proc    | 0.8424   | 0.4928  | 0.1979            | 0.6907 |

---

### Per-group behavior (Female → 0, Male → 1)

#### ADV in-proc
- **Selection rate:** 0 **0.158**, 1 **0.651** → DP gap **0.493** (very large).  
- **TPR (Recall):** 0 **0.667**, 1 **0.865** → EO gap **0.198** (notable disparity, males detected more often).  
- **FPR:** 0 **0.063**, 1 **0.240** (males much higher).  
- **Accuracy:** Female **0.895**, Male **0.829** → stronger performance for females.

---

### Implications
- **DP disparity is the worst among your ADV runs (0.493)**, showing males are far more frequently selected.  
- **EO gap is also higher (0.198)**, reflecting unequal recall between sexes.  
- **Accuracy (0.842)** is solid overall, but fairness outcomes are weaker compared to earlier ADV configurations.  
- **Interpretation:** This ADV setup improves neither DP nor EO compared to previous runs — fairness worsens, even though accuracy remains competitive.

---

In [24]:
# Grid-tune AIF360 AdversarialDebiasing for better DP/EO balance and print with report_model
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
from aif360.algorithms.inprocessing import AdversarialDebiasing

# small search over key knobs; widen if needed
ADV_GRID = dict(
    adversary_loss_weight=[0.02, 0.05, 0.1, 0.2, 0.3],
    num_epochs=[40, 60, 80],
    batch_size=[64, 128],
    classifier_num_hidden_units=[32, 64]  # size of main net
)

def run_adv(loss_w=0.1, epochs=50, bs=128, hidden=64, seed=42):
    tf.compat.v1.reset_default_graph()
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.set_random_seed(seed)
    sess = tf.compat.v1.Session()
    with sess.as_default():
        adv = AdversarialDebiasing(
            privileged_groups=privileged_groups,
            unprivileged_groups=unprivileged_groups,
            debias=True,
            scope_name=f"adv_w{loss_w}_e{epochs}_b{bs}_h{hidden}",
            adversary_loss_weight=loss_w,
            num_epochs=epochs,
            batch_size=bs,
            classifier_num_hidden_units=hidden,
            sess=sess
        )
        adv.fit(bld_tr)
        pred_te = adv.predict(bld_te)
        yhat = pred_te.labels.ravel().astype(int)
        scores = getattr(pred_te, "scores", None)
        if scores is None:
            scores = yhat.astype(float)
    sess.close()
    tf.compat.v1.reset_default_graph()
    return yhat, scores

# Build once (as you did)
bld_tr = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_train_ready).reset_index(drop=True),
                  pd.Series(y_train, name=label_name),
                  pd.Series(A_train, name=protected_attr)], axis=1),
    label_names=[label_name], protected_attribute_names=[protected_attr],
    favorable_label=favorable_label, unfavorable_label=unfavorable_label
)
bld_te = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_test_ready).reset_index(drop=True),
                  pd.Series(y_test, name=label_name),
                  pd.Series(A_test, name=protected_attr)], axis=1),
    label_names=[label_name], protected_attribute_names=[protected_attr],
    favorable_label=favorable_label, unfavorable_label=unfavorable_label
)

# Search & pick the best by minimizing (DP + EO) with an accuracy floor
best = None
acc_floor = 0.86  # keep close to your current accuracy; adjust as you like
results = []
for w in ADV_GRID["adversary_loss_weight"]:
    for e in ADV_GRID["num_epochs"]:
        for bs in ADV_GRID["batch_size"]:
            for h in ADV_GRID["classifier_num_hidden_units"]:
                yhat, scores = run_adv(w, e, bs, h)
                acc = accuracy_score(y_test, yhat)
                dp, eo = fair_metrics(y_test, yhat, A_test, scores, absolute=True)
                obj = dp + eo
                results.append((obj, acc, dp, eo, w, e, bs, h, yhat, scores))
                if (best is None or obj < best[0]) and acc >= acc_floor:
                    best = (obj, acc, dp, eo, w, e, bs, h, yhat, scores)

# Report best and (optionally) a few runners-up
if best is None:
    # fallback: take global best even if below floor
    best = sorted(results, key=lambda t: t[0])[0]

obj, acc, dp, eo, w, e, bs, h, yhat_best, scores_best = best
_ = report_model(
    f"ADV in-proc (best) w={w}, e={e}, b={bs}, h={h}",
    y_test, yhat_best, A_test, scores=scores_best,
    note=f"combined gap (DP+EO)={obj:.4f}; acc={acc:.4f}"
)

epoch 0; iter: 0; batch classifier loss: 0.849574; batch adversarial loss: 0.678034
epoch 1; iter: 0; batch classifier loss: 0.751289; batch adversarial loss: 0.679811
epoch 2; iter: 0; batch classifier loss: 0.687964; batch adversarial loss: 0.705974
epoch 3; iter: 0; batch classifier loss: 0.666234; batch adversarial loss: 0.680836
epoch 4; iter: 0; batch classifier loss: 0.656251; batch adversarial loss: 0.691837
epoch 5; iter: 0; batch classifier loss: 0.569755; batch adversarial loss: 0.664430
epoch 6; iter: 0; batch classifier loss: 0.582162; batch adversarial loss: 0.673232
epoch 7; iter: 0; batch classifier loss: 0.496106; batch adversarial loss: 0.715469
epoch 8; iter: 0; batch classifier loss: 0.465372; batch adversarial loss: 0.695565
epoch 9; iter: 0; batch classifier loss: 0.495062; batch adversarial loss: 0.654428
epoch 10; iter: 0; batch classifier loss: 0.590101; batch adversarial loss: 0.651189
epoch 11; iter: 0; batch classifier loss: 0.490156; batch adversarial loss:

,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.0625,0.666667,0.157895,0.894737
1,0.875000,0.1800,0.875000,0.636986,0.856164


Overall -> Accuracy: 0.8641 | DP diff: 0.4791 | EO diff: 0.2083 | combined gap (DP+EO)=0.6874; acc=0.8641


## ADV In-processing (tuned)

### Results overview
| Variant            | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|--------------------|---------:|--------:|------------------:|------:|
| ADV in-proc (best) | 0.8641   | 0.4791  | 0.2083            | 0.6874 |

---

### Per-group behavior (Female → 0, Male → 1)

#### ADV in-proc (best, w=0.05, e=80, b=128, h=32)
- **Selection rate:** 0 **0.158**, 1 **0.637** → DP gap **0.479** (very large).  
- **TPR (Recall):** 0 **0.667**, 1 **0.875** → EO gap **0.208** (large, males recalled much better).  
- **FPR:** 0 **0.063**, 1 **0.180** (males show much higher false positives).  
- **Accuracy:** Female **0.895**, Male **0.856** → both groups perform reasonably, females slightly better.

---

### Implications
- **DP disparity is high (0.479)** — males are still much more likely to be selected.  
- **EO disparity is also large (0.208)** — recall parity is not achieved, males benefit.  
- **Accuracy (0.864)** is strong, but fairness metrics worsen compared to earlier tuned ADV runs.  
- **Overall:** This ADV tuning favors accuracy but comes at the cost of both **DP** and **EO fairness**, leading to the **highest combined gap (≈0.687)** among ADV settings tested.

---

# Overall Comparison of Bias Mitigation Results

## Summary Table
| Model | Best Variant | Accuracy | DP diff | EO diff | DP+EO | Key Trade-off |
|-------|--------------|---------:|--------:|--------:|------:|---------------|
| **KNN** | Pre: Reweigh | 0.8261 | **0.3590** | **0.0208** | **0.3798** | Modest accuracy drop, best fairness |
| **DT**  | Post: EqOdds | 0.8152 | **0.2913** | 0.0938 | **0.3851** | Best DP+EO balance, small acc loss |
| **RF**  | Baseline | 0.8424 | 0.4243 | **0.0312** | **0.4555** | Strong EO parity, but large DP gap |
| **MLP** | Post: EqOdds | 0.7989 | **0.3133** | **0.0312** | **0.3445** | Largest fairness improvement, acc cost |
| **ADV (untuned)** | Baseline ADV | 0.8424 | 0.4928 | 0.1979 | 0.6907 | Accuracy good, fairness worst |
| **ADV (tuned)** | ADV best | 0.8641 | 0.4791 | 0.2083 | 0.6874 | Highest acc, but high disparities |

---

## Cross-Model Insights

### 1. **Best Fairness Outcomes**
- **MLP + Post: EqOdds** achieves the **lowest combined gap (DP+EO ≈ 0.345)**, drastically improving both DP and EO, though at the cost of accuracy (~0.799).  
- **KNN + Pre: Reweigh** is the next most fair, nearly eliminating EO disparity and reducing DP while retaining accuracy above 0.82.  
- **DT + Post: EqOdds** balances both DP and EO well, with the second-lowest combined disparity (~0.385).

### 2. **Accuracy–Fairness Trade-offs**
- **KNN** and **DT** show clear fairness gains with mitigation, but accuracy drops slightly compared to baseline.  
- **RF** remains unchanged regardless of technique: baseline already has excellent EO (0.031) but suffers from a persistent DP gap (0.424).  
- **MLP** sacrifices the most accuracy when fairness improves — post-processing reduces DP and EO dramatically but drops acc to ~0.80.  

### 3. **ADV In-processing**
- Both **untuned and tuned ADV** runs yield **high accuracy (0.842–0.864)** but perform poorly on fairness, with **DP ≈ 0.48–0.49** and **EO ≈ 0.20**.  
- Earlier ADV configurations were stronger on EO, but these updated settings show worsened disparities.  
- **Conclusion:** ADV in this configuration favors performance over fairness and is less effective than KNN/MLP mitigations.

---

## Overall Takeaways
- **Most fair approach overall:** **MLP with Post: Equalized Odds** (lowest DP+EO).  
- **Best compromise (fairness + accuracy):** **KNN with Pre: Reweigh**, which improves both fairness metrics without heavily sacrificing accuracy.  
- **Most stable model:** **RF baseline**, already strong in EO parity but resistant to fairness interventions.  
- **Weakest fairness outcomes:** **ADV in-processing**, which increases disparities despite solid accuracy.  

### Overall Observation
- **Pre-processing (Reweighing)** generally helps **KNN** and slightly improves fairness in **MLP**, but worsens DT.  
- **Post-processing (EqOdds)** is highly effective for **MLP** and **DT**, but ineffective for **KNN** and **RF**.  
- **ADV** in this setup is not competitive with the other strategies in terms of fairness.  

----